In [115]:
import pandas as pd
import numpy as np
from numpy import random
import gensim
from gensim import corpora
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize.toktok import ToktokTokenizer

import glob
from joblib import Parallel, delayed
import os

import itertools
import multiprocessing
from itertools import islice

cpu_count = multiprocessing.cpu_count()

DATA_FOLDER = '../data/'

In [116]:
all_docs = glob.iglob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)
# corpus_file = open(DATA_FOLDER + 'corpus.txt', 'w')
# for item in all_docs:
#     corpus_file.write("%s\n" % os.path.relpath(item, DATA_FOLDER))

In [117]:
# from nltk.tokenize.toktok import ToktokTokenizer 
# toktok = ToktokTokenizer()
# toktok.tokenize('устройство по п . 3 , отличать тем , что оно снабжать устанавливать в головка винт , в который размещать шарик и ролик .  . ')

In [118]:
stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'ко'])
stop_list.extend(list(string.punctuation))

punkts = [s for s in string.punctuation if s not in '.!?']


class Corpus(gensim.corpora.TextCorpus):
    def get_texts(self):
        cur_parent = ''
        for filename in self.input: # for each relevant file
            cur_dir = os.path.dirname(filename)
            if cur_dir != cur_parent:
                print(cur_dir)
                cur_parent = cur_dir
            yield tokenize(open(filename).read())


def tokenize(file_text):
    tokens = nltk.word_tokenize(file_text)

    tokens = [validate_word(word) for word in tokens if check_word(word)]
    tokens = filter(lambda s: s != '', tokens)
    
    return tokens

            
def validate_word(word):
    word = word.replace("«", "").replace("»", "").replace("`", '').replace(".", '')
    if len(word) == 1:
        return ''

    return word


def check_word(word):
    if word in stop_list:
        return False
    if any(char.isdigit() for char in word):
        return False
    
    return True

In [119]:
def grouper(n, iterable):
    it = iter(iterable)
    while True:
       chunk = tuple(islice(it, n))
       if not chunk:
           return
       yield chunk

In [138]:
dfolder = os.path.join(DATA_FOLDER, 'FIPS/Inventions patents_txt_output/0c/0c/')
all_docs = glob.iglob(dfolder + '*.txt', recursive=True)

parallelizer = Parallel(n_jobs=cpu_count)

# this iterator returns the functions to execute for each task
tasks_iterator = ( delayed(Corpus)(list_block) for list_block in grouper(cpu_count, all_docs) )
result = parallelizer( tasks_iterator )

corp1 = result[0]
merged_corpus = corp1
for cur_corp in result[1:]:
    cur_to_dict1 = corp1.dictionary.merge_with(cur_corp.dictionary)
    merged_corpus = itertools.chain(merged_corpus, cur_to_dict1[cur_corp])
    
corpora.MmCorpus.serialize(os.path.join(DATA_FOLDER, 'corpus.mm'), merged_corpus)


../data/FIPS/Inventions patents_txt_output/0c/0c
../data/FIPS/Inventions patents_txt_output/0c/0c
../data/FIPS/Inventions patents_txt_output/0c/0c
../data/FIPS/Inventions patents_txt_output/0c/0c
../data/FIPS/Inventions patents_txt_output/0c/0c
../data/FIPS/Inventions patents_txt_output/0c/0c


In [131]:
for i in merged_corpus:
    print(i)

../data/FIPS/Inventions patents_txt_output/0c/0c
[(7, 8), (34, 2), (35, 2), (39, 3), (40, 1), (50, 1), (52, 3), (53, 1), (55, 4), (56, 2), (63, 1), (64, 6), (66, 17), (77, 1), (78, 6), (88, 1), (89, 1), (92, 2), (95, 10), (97, 1), (109, 1), (118, 1), (120, 2), (126, 12), (136, 7), (173, 20), (174, 3), (176, 1), (179, 2), (184, 1), (194, 16), (205, 1), (207, 1), (223, 7), (227, 7), (237, 1), (247, 3), (248, 3), (259, 2), (262, 8), (264, 1), (271, 6), (275, 2), (285, 3), (306, 49), (326, 1), (328, 3), (339, 3), (365, 3), (384, 1), (391, 6), (392, 24), (394, 1), (399, 5), (405, 5), (411, 1), (417, 4), (419, 2), (422, 1), (424, 5), (444, 1), (447, 5), (448, 4), (457, 2), (458, 1), (459, 2), (467, 1), (469, 33), (479, 2), (480, 1), (481, 1), (501, 1), (504, 1), (507, 28), (510, 28), (514, 39), (518, 1), (521, 6), (523, 11), (526, 11), (566, 31), (580, 5), (583, 27), (591, 7), (592, 1), (617, 33), (628, 2), (630, 1), (632, 2), (634, 7), (638, 2), (650, 3), (651, 34), (660, 2), (662, 1), (671

In [95]:
list_block

NameError: name 'list_block' is not defined

In [50]:
merged_corpus

In [27]:
result[0]

In [23]:
result

In [8]:
result

In [13]:
mycorpus = Corpus(all_docs)
corpora.MmCorpus.serialize(os.path.join(DATA_FOLDER, 'corpus.mm'), mycorpus)

../data/FIPS/Inventions patents_txt_output/0c/0c


In [8]:
mycorpus.dictionary.token2id

{'концентрат': 0,
 'смазочно-охлаждающей': 1,
 'жидкость': 2,
 'механический': 3,
 'обработка': 4,
 'металл': 5,
 'использование': 6,
 'содержать': 7,
 'мас': 8,
 'минеральный': 9,
 'масло': 10,
 'продукт': 11,
 'конденсация': 12,
 'ненасыщать': 13,
 'жирный': 14,
 'кислота': 15,
 'мольный': 16,
 'соотношение': 17,
 'растительный': 18,
 'фракция': 19,
 'гликоли': 20,
 'полиоксиэтилированный': 21,
 'алкилфенол': 22,
 'взаимодействие': 23,
 'кислый': 24,
 'эфир': 25,
 'борный': 26,
 'алканоламин': 27,
 'двузамещать': 28,
 'алифатический': 29,
 'фосфорный': 30,
 'пеногаситель': 31,
 'вода': 32,
 'остальное': 33,
 'технический': 34,
 'результат': 35,
 'улучшение': 36,
 'санитарно-гигиенических': 37,
 'характеристика': 38,
 'повышение': 39,
 'устойчивость': 40,
 'влияние': 41,
 'солея': 42,
 'жесткость': 43,
 'сохранение': 44,
 'высокий': 45,
 'антикоррозионный': 46,
 'свойство': 47,
 'биостойкость': 48,
 'табл': 49,
 'отличать': 50,
 'дополнительно': 51,
 'качество': 52,
 'следующий': 53,


In [10]:
mycorpus.dictionary.doc2bow(['концентрат', 'металл'])

[(0, 1), (5, 1)]

In [28]:
wv = gensim.models.KeyedVectors.load_word2vec_format(
    "~/Documents/wordvecs/ruwikiruscorpora_0_300_20.bin",
    binary=True)
wv.init_sims(replace=True)

In [29]:
from itertools import islice
list(islice(wv.vocab, 13000, 13020))

['кампус_NOUN',
 'макдональд_NOUN',
 'девятнадцатый_ADJ',
 'электротехнический_ADJ',
 'проникнутый_ADJ',
 'полушубок_NOUN',
 'клавишный_ADJ',
 'годичный_ADJ',
 'знаменательный_ADJ',
 'горох_NOUN',
 'задрожать_VERB',
 'вавилонский_ADJ',
 'скорпион_NOUN',
 'сворачивать_VERB',
 'недалекий_ADJ',
 'оглядеться_VERB',
 'четырехлетний_ADJ',
 'мамочка_NOUN',
 'котелок_NOUN',
 'news_NOUN']

In [31]:
wv.most_similar(positive=['силос_NOUN'], restrict_vocab=100000, topn=30)[0:20]

[('силосный_ADJ', 0.657220721244812),
 ('корнеплод_NOUN', 0.5938321948051453),
 ('компост_NOUN', 0.5927795767784119),
 ('корм_NOUN', 0.5741930603981018),
 ('удобрение_NOUN', 0.5622274875640869),
 ('ботва_NOUN', 0.5541197061538696),
 ('аммиачный::селитра_NOUN', 0.5507166385650635),
 ('зерно_NOUN', 0.5447162389755249),
 ('бурт_NOUN', 0.5432900190353394),
 ('комбикорм_NOUN', 0.5354993343353271),
 ('жмых_NOUN', 0.5318485498428345),
 ('овощехранилище_NOUN', 0.5261991024017334),
 ('навоз_NOUN', 0.5203781127929688),
 ('отруби_NOUN', 0.5183936953544617),
 ('сушилка_NOUN', 0.5149815678596497),
 ('птичий::помет_NOUN', 0.514065682888031),
 ('клубень_NOUN', 0.5139718651771545),
 ('кукуруза_NOUN', 0.507766604423523),
 ('гранулированный_ADJ', 0.5018250346183777),
 ('брикет_NOUN', 0.4995535910129547)]

### для нового тектса нужно обязательно: токенизировать, разбить на предложения, лемматизировать и разметить по частям речи при помощи Mystem

In [34]:
!./mystem -i --format json test.txt

[{"analysis":[{"lex":"программа","gr":"S,жен,неод=им,ед"}],"text":"Программа"},{"analysis":[],"text":"MyStem"},{"analysis":[{"lex":"производить","gr":"V=непрош,ед,изъяв,3-л,несов,пе"}],"text":"производит"},{"analysis":[{"lex":"морфологический","gr":"A=вин,ед,полн,муж,неод"},{"lex":"морфологический","gr":"A=им,ед,полн,муж"}],"text":"морфологический"},{"analysis":[{"lex":"анализ","gr":"S,муж,неод=вин,ед"},{"lex":"анализ","gr":"S,муж,неод=им,ед"}],"text":"анализ"},{"analysis":[{"lex":"текст","gr":"S,муж,неод=род,ед"}],"text":"текста"},{"analysis":[{"lex":"на","gr":"PR="},{"lex":"на","gr":"PART="}],"text":"на"},{"analysis":[{"lex":"русский","gr":"A=пр,ед,полн,муж"},{"lex":"русский","gr":"A=пр,ед,полн,сред"},{"lex":"русский","gr":"S,муж,од=пр,ед"}],"text":"русском"},{"analysis":[{"lex":"язык","gr":"S,муж,неод=пр,ед"},{"lex":"язык","gr":"S,муж,од=пр,ед"}],"text":"языке"},{"analysis":[{"lex":"она","gr":"SPRO,ед,3-л,жен=им"}],"text":"Она"},{"analysis":[{"lex":"уметь","gr":"V,несов,нп=непрош,ед

In [17]:
30*24*60.

43200.0